## 3. Подготовка PyTorch Geometric графа (Data object)

Этот блок:

1. Превращает node_df в:
    - матрицу признаков x
    - вектор таргетов y

2. Превращает edge_df в:
    - edge_index
    - edge_attr

3. Собирает всё в один torch_geometric.data.Data

In [2]:
# Подготовка графа для PyTorch Geometric

import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler


def prepare_pyg_graph(node_df: pd.DataFrame, edge_df: pd.DataFrame, target_col='churn_rate', 
                     src_col='src_node', dst_col='dst_node'):
    """
    Превращает node_df и edge_df в Data объект PyTorch Geometric.
    
    Аргументы:
        node_df: DataFrame с информацией о вершинах, index = node_id
        edge_df: DataFrame с информацией о рёбрах, содержит столбцы src_col и dst_col
        target_col: имя столбца с целевой переменной
        src_col: имя столбца с исходной вершиной в edge_df
        dst_col: имя столбца с конечной вершиной в edge_df
    
    Возвращает:
        graph: Data (x, edge_index, edge_attr, y)
        node_id_map: dict node_id -> index
    """

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 1. Мэппинг node_id → индекс
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    node_ids = list(node_df.index)
    node_id_map = {nid: i for i, nid in enumerate(node_ids)}

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 2. Разбор числовых фичей нод
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

    # Вытащим embedding и временно уберём
    text_emb = node_df['text_embedding'].apply(lambda x: np.array(x, dtype=float))
    node_df_wo_emb = node_df.drop(columns=['text_embedding'])

    # Категориальные фичи кодируем числом
    node_cat_cols = []
    for col in node_df_wo_emb.columns:
        if node_df_wo_emb[col].dtype == 'object':
            node_cat_cols.append(col)
            node_df_wo_emb[col] = node_df_wo_emb[col].astype('category').cat.codes

    # Все числовые колонки (исключая целевую)
    numeric_cols = [c for c in node_df_wo_emb.columns if c != target_col]

    # Выделяем матрицу фичей нод
    X_numeric = node_df_wo_emb[numeric_cols].astype(float).values

    # Скейлинг
    scaler = StandardScaler()
    X_numeric = scaler.fit_transform(X_numeric)

    # Добавляем text embedding
    X_emb = np.vstack(text_emb.values)
    X = np.concatenate([X_numeric, X_emb], axis=1)

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 3. Таргет y
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    y = torch.tensor(node_df_wo_emb[target_col].astype(float).values, dtype=torch.float)

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 4. Графовые данные: edge_index
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # Проверяем, что все вершины в edge_df существуют в node_df
    all_src_nodes = set(edge_df[src_col])
    all_dst_nodes = set(edge_df[dst_col])
    all_edge_nodes = all_src_nodes.union(all_dst_nodes)
    
    missing_nodes = all_edge_nodes - set(node_ids)
    if missing_nodes:
        raise ValueError(f"В edge_df есть вершины, отсутствующие в node_df: {list(missing_nodes)[:5]}...")
    
    # Преобразуем имена вершин в индексы
    sources = [node_id_map[s] for s in edge_df[src_col]]
    targets = [node_id_map[d] for d in edge_df[dst_col]]

    edge_index = torch.tensor([sources, targets], dtype=torch.long)

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 5. edge_attr (фичи связей)
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    edge_df_local = edge_df.copy()
    
    # Удаляем столбцы src и dst, так как они уже использованы для edge_index
    edge_df_local = edge_df_local.drop(columns=[src_col, dst_col])
    
    # категориальные в связях
    edge_cat_cols = []
    for col in edge_df_local.columns:
        if edge_df_local[col].dtype == 'object':
            edge_cat_cols.append(col)
            edge_df_local[col] = edge_df_local[col].astype('category').cat.codes

    edge_X = edge_df_local.astype(float).values
    edge_X = torch.tensor(edge_X, dtype=torch.float)

    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    # 6. Финальный граф
    # :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
    graph = Data(
        x=torch.tensor(X, dtype=torch.float),
        edge_index=edge_index,
        edge_attr=edge_X,
        y=y
    )

    # Информация о структуре фичей
    feature_info = {
        'node_features': {
            'numeric_features': numeric_cols,  # Числовые фичи (до скейлинга)
            'categorical_features': node_cat_cols,  # Категориальные фичи
            'embedding_dim': X_emb.shape[1],  # Размерность эмбеддинга
            'total_dimension': X.shape[1],  # Общая размерность
            'feature_order': numeric_cols + ['text_embedding'],  # Порядок фичей
            'scaler': scaler  # Объект скейлера
        },
        'edge_features': {
            'columns': list(edge_df_local.columns),
            'dimension': edge_X.shape[1]
        }
    }
    
    print("=" * 60)
    print("СТРУКТУРА ФИЧЕЙ УЗЛОВ:")
    print(f"1. Числовые фичи ({len(numeric_cols)}):")
    for i, feat in enumerate(numeric_cols):
        print(f"   {i}: {feat}")
    
    print(f"\n2. Категориальные фичи ({len(node_cat_cols)}):")
    for cat in node_cat_cols:
        print(f"   - {cat}")
    
    print(f"\n3. Текстовая эмбеддинг (размерность: {X_emb.shape[1]})")
    
    print(f"\n4. ОБЩАЯ СТРУКТУРА (всего {X.shape[1]} фичей):")
    print(f"   [0-{len(numeric_cols)-1}]: Числовые фичи")
    print(f"   [{len(numeric_cols)}-{X.shape[1]-1}]: Текстовый эмбеддинг")
    print("=" * 60)

    return graph, node_id_map, scaler, feature_info




In [3]:
# edge_df = pd.read_csv("../../data/edge_df.csv")
# node_df = pd.read_parquet("../../data/node_semantic_df.parquet")

# display(edge_df)
# display(node_df)

# graph, node_id_map, scaler, feature_info = prepare_pyg_graph(node_df, edge_df)

# print(graph)

# Использование функции
edge_df = pd.read_csv("../../data/edge_df.csv")
node_df = pd.read_parquet("../../data/node_semantic_df.parquet")

# Убедимся, что node_df имеет правильный индекс
if 'node_id' in node_df.columns and node_df.index.name != 'node_id':
    # Если node_id есть в столбцах, но не в индексе
    node_df = node_df.set_index('node_id')

display(edge_df.head())
display(node_df.head())

graph, node_id_map, scaler, feature_info = prepare_pyg_graph(
    node_df, 
    edge_df,
    target_col='churn_rate',
    src_col='src_node',
    dst_col='dst_node'
)

print(f"Граф создан: {graph}")
print(f"Количество вершин: {graph.num_nodes}")
print(f"Количество рёбер: {graph.num_edges}")
print(f"Размер node features: {graph.x.shape}")
print(f"Размер edge features: {graph.edge_attr.shape}")
print(feature_info)

,src_node,dst_node,transition_count,unique_users,unique_sessions,avg_time_between_nodes_sec,median_time_between_nodes_sec,bounce_rate_on_target,churn_rate_on_target
0,0072f89b60d46ef6f2094949d8831f13,0072f89b60d46ef6f2094949d8831f13,13271,1409,353,15.936101,6.0,0.138347,0.020420
1,0072f89b60d46ef6f2094949d8831f13,02b207cc24a78c1942161bafc72fe532,290,255,85,43.103448,25.0,0.755172,0.134483
2,0072f89b60d46ef6f2094949d8831f13,0ab7553a46130fe3b64fa66ae66e6ad1,35,30,28,46.628571,30.0,0.800000,0.028571
3,0072f89b60d46ef6f2094949d8831f13,137091633a6334ce94ced79cab6ec771,5,5,5,79.200000,71.0,0.200000,0.000000
4,0072f89b60d46ef6f2094949d8831f13,18b0d595350748c3fd82491de6631219,23,15,11,84.086957,62.0,0.391304,0.086957


,screen,feature,action,total_visits,sessions_with_node,avg_visits_per_session,median_visits_per_session,avg_session_length,avg_time_on_page_seconds,median_time_on_page_seconds,...,churn_count,churn_rate,bounce_count,bounce_rate,avg_age,male_ratio,device_vendor_top,device_type_top,os_top,text_embedding
node_id,,,,,,,,,,,,,,,,,,,,,
0072f89b60d46ef6f2094949d8831f13,Важное,Просмотр уведомления,Тап на уведомление,7489,4609,1.624864,1.0,4.175092,30.737356,12.0,...,389,0.051943,2977,0.397516,46.105622,0.464548,Apple,phone,Android,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
02b207cc24a78c1942161bafc72fe532,Еще,Переход в раздел 'Опросы и собрания собственни...,Тап на кнопку 'Опросы и собрания собственников',4167,3658,1.139147,1.0,4.681520,63.280683,24.0,...,246,0.059035,2291,0.549796,46.174706,0.438445,Apple,phone,Android,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
03fa5765e1bee3c6ba8b9a139635c46a,Новый адрес,Активация гостевого доступа,Тап на кнопку 'Активировать',9,5,1.800000,1.0,4.400000,7.000000,6.0,...,0,0.000000,3,0.333333,49.444444,0.777778,Sony,phone,Android,"[0.0, 0.5459835934470357, 0.0, 0.0, 0.0, 0.0, ..."
05aa62cfe2beb31d4ecc652cddec5689,Объявления,Редактирование опубликованного объявления,Тап на кнопку 'Редактировать',3,3,1.000000,1.0,7.000000,62.000000,62.0,...,0,0.000000,1,0.333333,48.666667,0.333333,Apple,phone,iOS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
061da77ad7d449a342174810fbf72350,Гостевой доступ,Раскрытие вкладки 'Архив',Тап на кнопку 'Архив',6,6,1.000000,1.0,16.333333,2.500000,2.0,...,0,0.000000,2,0.333333,32.333333,0.500000,Samsung,phone,Android,"[0.0, 0.0, 0.0, 0.0, 0.5293365357504765, 0.0, ..."


СТРУКТУРА ФИЧЕЙ УЗЛОВ:
1. Числовые фичи (20):
   0: screen
   1: feature
   2: action
   3: total_visits
   4: sessions_with_node
   5: avg_visits_per_session
   6: median_visits_per_session
   7: avg_session_length
   8: avg_time_on_page_seconds
   9: median_time_on_page_seconds
   10: avg_session_duration_seconds
   11: repeat_ratio
   12: churn_count
   13: bounce_count
   14: bounce_rate
   15: avg_age
   16: male_ratio
   17: device_vendor_top
   18: device_type_top
   19: os_top

2. Категориальные фичи (6):
   - screen
   - feature
   - action
   - device_vendor_top
   - device_type_top
   - os_top

3. Текстовая эмбеддинг (размерность: 64)

4. ОБЩАЯ СТРУКТУРА (всего 84 фичей):
   [0-19]: Числовые фичи
   [20-83]: Текстовый эмбеддинг
Граф создан: Data(x=[134, 84], edge_index=[2, 1223], edge_attr=[1223, 7], y=[134])
Количество вершин: 134
Количество рёбер: 1223
Размер node features: torch.Size([134, 84])
Размер edge features: torch.Size([1223, 7])
{'node_features': {'numeric_featur

### Что мы получили

Теперь у нас полностью готов объект Data, который можно отправлять в GraphSAGE/GCN/GAT:
- graph.x — признаки нод
- graph.edge_index — структура графа
- graph.edge_attr — признаки связей
- graph.y — churn_rate вашей ноды

### Сохранить граф в файл

In [30]:
import pickle
import torch
from torch_geometric.data import Data

# Сохранение всей структуры
def save_graph_data(filename, graph, node_id_map, scaler, numeric_cols):
    data_to_save = {
        'graph': graph,
        'node_id_map': node_id_map,
        'scaler': scaler,
        'numeric_cols': numeric_cols
    }
    
    with open(filename, 'wb') as f:
        pickle.dump(data_to_save, f)
    print(f"Данные сохранены в {filename}")

# Сохраняем
save_graph_data('../../data/graph_data.pkl', graph, node_id_map, scaler, numeric_cols)

Данные сохранены в ../../data/graph_data.pkl
